# 5章リカレントニューラルネットワーク(RNN)

- **これまで見てきたニューラルネットワーク**
    - **フィードフォワードと呼ばれるタイプ**
        - 意味
            - 流れが一方向のネットワーク
                - 具体的に言うと、入力信号が次の層（隠れ層）へ信号を伝達し、信号を受け取った層はその次の層へ伝達し、そしてまた次の層へ……といったように一方向だけの信号伝達を行う
        - 問題点
            - 時系列データをうまく扱えないことです。
                - 単純なフィードフォワード・ネットワークでは、時系列データの性質（パターン）を十分に学習することができない

    ↓ その問題を解消するのが

- **これから見ていくニューラルネットワーク**
    - **リカレントニューラルネットワーク（Recurrent Neural Network）、略してRNN**
        - 本章では、フィードフォワード・ネットワークの問題点を指摘し、RNN がその問題を見事に解決できることを説明します

## 5.1 確率と言語モデル

ここではRNN の話を始める前の準備として、前章のword2vec の復習から始めます。また、自然言語に関する現象を「確率」を使って記述し、最後に、言語を確率として扱う「言語モデル」について説明します。

### 5.1.1 word2vec を確率の視点から眺める

- 前章のword2vec のCBOW モデルの復習
    - ここでは、$w_1,w_2, . . . ,w_T$ という単語の列で表されるコーパスを考える。
    - そして、$t$番目の単語を「ターゲット」として、その前後の単語（$t−1$番目と$t+1$番目）を「コンテキスト」として扱う。
    - 本書では、ターゲットは「中央の単語」を、コンテキストはターゲットの「周囲の単語」を指す言葉として用いる。

![5_1](fig5/5_1.png)

- $w_t−1$ と $w_t+1$ が与えられたとき、ターゲットがw_t となる確率を数式で表してみましょう。これは以下のように書くことができます。
    -   式(5.1) $$ P(w_t|w_{t−1},w_{t+1}) $$
- CBOWモデルは、式(5.1) の事後確率をモデル化します。この事後確率は、「$w_t−1$と$w_t+1$ が与えられたとき、$w_t$ が起こる確率」を表します。これがウィンドウサイズが1 のときのCBOWモデルです。

- ところで、私たちはこれまでコンテキストとして左右対称のウィンドウを考えてきました。ここではコンテキストとして、左のウィンドウだけに限定してみたいと思います。たとえば、図5-2 のようなケースを考えてみます。![5_2](fig5/5_2.png)

- 図5-2のように、ここで私たちは左側の2つの単語だけをコンテキストとして考えることにします。
そうすると、CBOW モデルが出力する確率は式(5.2) のようになります。
    -   式(5.2) $$ P(w_t|w_{t−2},w_{t-1}) $$
- word2vec のコンテキストのウィンドウサイズは、ハイパーパラメータであり、任意の値に設定できます。ここでは、ウィンドウサイズを「左を2 単語、右を0 単語」として左右非対称で設定しました。このように設定した理由は、後ほど説明する「言語モデル」を見越してのことです。

- さて、式(5.2) の表記を用いれば、CBOW モデルが扱う損失関数は、式(5.3) のように書くことができます。ちなみに式(5.3) は、交差エントロピー誤差により導かれた結果です（詳細は「1.3.1 損失関数」を参照）。
$$ L = −logP(w_t|w_{t−2},w_{t−1}) $$

- CBOW モデルの学習で行うことは、式(5.3) で表される損失関数――正確には、コーパス全体の損失関数の総和――をできる限り小さくする重みパラメータを見つけることです。もしそのような重みパラメータが見つかれば、CBOWモデルはコンテキストからターゲットをより正しく推測できるようになります。
- このようにCBOWモデルの学習の本来の目的は、コンテキストからターゲットを正しく推測できるようになることです。この目的を達成するために学習を進めると、（その副産物として）単語の意味がエンコードされた「単語の分散表現」が得られました。
- それでは、CBOW モデルの本来の目的であった「コンテキストからターゲットを推測すること」は、何かに利用できるのでしょうか？ 式(5.2) で表される確率 $ P(w_t|w_{t−2},w_{t−1}) $ は、実用的な用途で利用できるのでしょうか？ そこに「言語モデル」が関係します。

### 5.1.2 言語モデル

- 言語モデル（Language Model）は、単語の並びに対して確率を与えます。単語の並びに対して、それがどれだけ起こりえるのか――それがどれだけ自然な単語の並びであるのか――ということを確率で評価するのです。
    - たとえば、「you say goodbye」という単語の並びには高い確率（たとえば0.092）を出力し、「you say good die」という単語の並びには低い確率（たとえば0.0000000000032）を出力する、というようなことを言語モデルは行います。

- この言語モデルは、さまざまなアプリケーションで利用することができます。機械翻訳や音声認識などが、その代表例です。
    - たとえば、ある音声認識システムの場合、人の発話からいくつかの文章を候補として生成するでしょう。その場合、言語モデルを使えば、候補となる文章が「文章として自然であるかどうか」という基準でランク付けすることができます。

- また、言語モデルは新しい文章を生成する用途にも利用できます。というのも、言語モデルは単語列の自然さを確率的に評価できるため、その確率分布に従って単語を“紡ぎ出す”（サンプリングする）ことができるのです。なお、言語モデルを使った文章生成は「7 章RNN による文章生成」で取り上げます。

- それでは、数式を使って言語モデルを記述しましょう。ここでは、$ w_1, . . . ,w_m $という$m$個の単語からなる文章について考えます。このとき、$w_1, . . . ,w_m$という順序で単語が出現する確率は、$P(w_1, . . . ,w_m)$ で表されます。この確率は複数の事象が同時に起こる確率であるため、同時確率と呼ばれます。この同時確率の$P(w_1, . . . ,w_m)$ ですが、これは事後確率を使って次のように分解して書くことができます。

$$
\begin{align}
P(w_1,...,w_m) &= P(w_m|w_1,...,w_{m−1})P(w_{m−1}|w_1,...,w_{m−2})...P(w_3|w_1,w_2)P(w_2|w_1)P(w_1) \\
               &= \prod_{t=1}^{m}P(w_t|w_1,...,w_{t−1}) \\
\end{align}
$$

- 式(5.4) の $\prod$ （パイ）という記号は、すべての要素をかけ合わせる「総乗」を表します（総和を表すのが $\sum$ （シグマ）であるのに対して、総乗の記号は$\prod$ （パイ）で表されます）。式(5.4) で示すとおり、同時確率は事後確率の総乗で表すことができます。

- 式(5.4) の結果は、確率の乗法定理から導くことができます。ここでは、少し時間をとって乗法定理について説明し、式(5.4) の導出する過程を見ていきたいと思います。早速、確率の乗法定理についてですが、これは次の式で表されます。
    - 式(5.5) $$ P(A,B) = P(A|B)P(B)$$

- この式(5.5) で表される乗法定理が、確率論において最も重要な定理です。この定理が意味することは、「A とB の両方が起こる確率$P(A,B)$」は、「B が起こる確率$P(B)$」と「B が起こった後にA が起こる確率$P(A|B)$」をかけ合わせたものになるということです（この解釈は自然なものに感じられるでしょう）。
- この乗法定理を使えば、m 個の単語の同時確率$P(w_1, . . . ,w_m)$ を事後確率で表すことができます。このとき行う式変形を分かりやすく示すと、次のようになります。
![5_6](fig5/5_6.png)

- ここでは、$w_1,...,w_{m−1}$ をまとめてA という記号で表します。そうすると、乗法定理に従うことで式(5.6) が導かれます。続いて、この$A（w_1, . . . ,w_{m−1}）$に対して、再び同じ式変形を行います。
![5_7](fig5/5_7.png)

- このように単語の並びをひとつずつ小さくしながら、そのつど事後確率に分解していきます。後は同様の手順を繰り返すことで、式(5.4) を導くことができるのです。さて、式(5.4) が示すように、目的とする同時確率の$P(w_1,...,w_m)$ は、事後確率の総乗である
$\prod P(w_t|w_1, ...w_{t−1})$ 
- によって表すことができます。ここで注目すべきは、その事後確率は、対象の単語より左側のすべての単語をコンテキスト（条件）としたときの確率であるということです。これは図で表すと、図5-3 のようになります。
![5_3](fig5/5_3.png)

- ここまでの話をまとめると、私たちの目標は$P(w_t|w_1,...,w_{t−1})$ という確率を得ることです。その確率が計算できれば、言語モデルの同時確率$P(w_1,...,w_m)$ を求めることができます。

### 5.1.3 CBOWモデルを言語モデルに?

- それでは、word2vec のCBOWモデルを、（無理やり）言語モデルに適用するにはどうしたらよいでしょうか？ それには、コンテキストのサイズをある値に限定することで、近似的に表すことができます。これは数式で表すと次のようになります。
![5.8](fig5/5_8.png)
- ここでは、コンテキストを左側の2 つの単語に限定することにします。そうすれば、CBOWモデルによって―― CBOWモデルの事後確率によって――、近似的に表すことができます。
- 式(5.8) では、コンテキストとして2 つの単語を用いる例を示しましたが、このコンテキストのサイズは任意の長さに設定できます（たとえば、5 や10 など）。しかし、任意の長さに設定できるとはいえ、それはある長さに“固定” しなければなりません。たとえば、左側の10 個の単語をコンテキストとしてCBOW モデルを作るにしても、そのコンテキストよりもさらに左側にある単語の情報は無視されてしまいます。これが問題になるのは、たとえば図5-4 のような例です。
![5.4](fig5/5_4.png)
- 図5-4 の問題では、「Tom が部屋でテレビを見ていて、Mary がその部屋に入って来た」とあります。その文脈（コンテキスト）を踏まえると、Mary が「Tom」（もしくは「彼」）にあいさつをするというのが正解になります。ここで正しい答えを得るには、例文の「？」から18 個も前に登場するTom を記憶しておく必要があるのです。もしCBOW モデルのコンテキストが10 個までだったとしたら、この問題を正しく答えることはできないでしょう。
- それでは、CBOWモデルのコンテキストのサイズを20 や30 といったように大きくすれば問題は解決するのでしょうか。確かに、CBOWモデルのコンテキストのサイズはいくらでも大きくすることはできます。しかし、CBOWモデルではコンテキスト内の単語の並びが無視されるという問題があります。
- コンテキストの単語の並びが無視される問題について、ひとつ具体例を出して説明しましょう。たとえば、コンテキストとして2 個の単語を扱う場合、CBOWモデルではその2 個の単語ベクトルの「和」が中間層に来ます。図で表せば、図5-5 の左図のようになります。
![5_5](fig5/5_5.png)

- 図5-5 の左図に示すように、CBOWモデルの中間層では単語ベクトルの和が求められるため、コンテキストの単語の並び方は無視されます。たとえば(you, say) と(say, you) というコンテキストが同じものとして扱われます。本来であれば、コンテキストの単語の並びも考慮したモデルが望まれることでしょう。これを達成するためには、図5-5 の右図に示すように、コンテキストの単語ベクトルを中間層において「連結（concatenate）」することが考えられます。実際、Neural Probabilistic Language Model [28] で提案されたモデルは、そのようなアプローチをとっています（モデルの詳細は論文[28] を参照）。しかし、連結するアプローチをとったとしたら、コンテキストのサイズに比例して重みパラメータが増えることになります。もちろん、そのようなパラメータの増加は歓迎されません。

- それでは、ここで指摘した問題を解決するにはどうしたらよいでしょうか？ お察しのとおり、ここで登場するのがリカレントニューラルネットワーク、略してRNNです。RNN は、コンテキストがどれだけ長くても、そのコンテキストの情報を記憶するメカニズムを持ちます。そのためRNN を使えば、どんなに長い時系列データにも立ち向かうことができるのです！ それでは続いて、この素晴らしきRNN の世界をじっくり堪能していきたいと思います。
- word2vec は、単語の分散表現を獲得することを目的に考案された手法です。そのため、それが言語モデルとして使用されることは（通常は）ありません。ここではRNN の魅力を引き出すために、word2vec のCBOW モデルを無理やり言語モデルに適用するような話の展開にしました。ちなみに、word2vecは2013年、この後で見ていくRNN による言語モデルは2010 年にTomas Mikolov氏らのチームによって、それぞれ提案されました。RNN による言語モデルでも単語の分散表現を獲得できたのですが、語彙数増加への対応や単語の分散表現の“質” の向上のために、word2vec が提案されたというのが、これらの研究の背景にあります。